In [1]:
import tensorflow as tf
import numpy as np
import glob, os
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
def getData(dir):
    all_data = []
    for file in sorted(glob.glob(dir)):
        datasets = []
        for f in open(file, 'r'):
            f = f.replace(']', '').replace('[', '').replace('\n','')
            (u, v, w) = f.split(',')
            datasets.append([int(u), int(v), float(w)])

        all_data.append(datasets)
    return all_data

In [3]:
dir = 'sequence/*'
all_data = getData(dir)

In [21]:
# 8:2 sperate
x_train, x_test = train_test_split(all_data, test_size = 0.2)

sequence_length_tr = []
sequence_length_te = []
for x in x_train:
    sequence_length_tr.append(len(x))
for x in x_test:
    sequence_length_te.append(len(x))
max_squence_length = max(max(sequence_length_tr), max(sequence_length_te))

In [26]:
# timesteps 맞춰주기 위해서 padding 추가
x_train = sequence.pad_sequences(x_train, padding='post', maxlen = max_squence_length)
x_test = sequence.pad_sequences(x_test, padding='post', maxlen = max_squence_length)

In [27]:
n_features = 3
batch_size = 32
epochs = 1000
n_neurons = 5

In [28]:
X = tf.placeholder(tf.float32, [None, max_squence_length, n_features])
seq_length = tf.placeholder(tf.int32, [None])

In [29]:
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

W0910 14:23:14.763907 139910020839168 deprecation.py:323] From <ipython-input-29-4ca4123af35d>:1: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0910 14:23:14.768298 139910020839168 deprecation.py:323] From <ipython-input-29-4ca4123af35d>:2: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0910 14:23:14.836268 139910020839168 deprecation.py:506] From /home/minji/anaconda3/envs/graph/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance 

In [31]:
x_train = np.array(x_train)
sequence_length = np.array(sequence_length_tr)
sequence_length.shape

(540,)

In [32]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X:x_train, seq_length: sequence_length})

In [33]:
print('outputs_val:{}\n{}'.format(outputs_val.shape, outputs_val))

outputs_val:(540, 41, 5)
[[[ 0.7370587  -0.7085977  -0.9962466   0.9834519  -0.609181  ]
  [ 0.6093976  -0.9146167  -0.9795989   0.9978158  -0.77537805]
  [ 0.98437685 -0.0146181  -0.9953956   0.9197622   0.7821091 ]
  ...
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.79109913 -0.9991193  -0.9997015   0.9997609   0.99710506]
  [ 0.6815129  -0.95549065 -0.65434617  0.9930543   0.15040866]
  [ 0.7973921  -0.9998274  -0.9993251   0.9999819   0.9951826 ]
  ...
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.99772143 -0.7805537  -0.9999966   0.9961537   0.97322416]
  [ 0.9983949   0.10280859 -0.9996192   0.9837928   0.30534738]
  [ 0.9890568  -0.42574787 -0.99881876  0.98929507 -0.21550775]